In [5]:
import transformers
import tensorflow

2024-12-26 18:51:57.537841: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-26 18:51:57.538373: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-26 18:51:57.541031: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-26 18:51:57.547770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735210317.558097   77570 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735210317.56

In [6]:
f = transformers.pipeline('sentiment-analysis')

f(['123'])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [16]:
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")


In [17]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig
import torch

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

2024-12-28 19:40:02.925712: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-28 19:40:03.132095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735386003.243526    5599 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735386003.273582    5599 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-28 19:40:03.490794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [18]:
model = GPT2LMHeadModel(config)
# for i in dir(model):
#     print(i)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.2M parameters


In [19]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [21]:
out = data_collator([train["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['Unnamed: 0', 'title', 'TL', 'ML', 'Input', 'Output', 'Note', 'Statement', 'contest', 'index', 'tags', 'rating', 'sample-input', 'sample-output', 'contest-name']

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

train = load_dataset('csv', data_files = 'train_data.csv')
import pandas as pd

f = pd.read_csv('train_data.csv')
# f = f['Statement']
f = f[['Statement','tags']].to_dict()
print(f.keys())

train = Dataset.from_dict(f)

# ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train", streaming = True)
# ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="train", streaming = True)

# raw_datasets = DatasetDict(
#     {
#         "train": ds_train,  # .shuffle().select(range(50000)),
#         "valid": ds_valid,  # .shuffle().select(range(500))
#     }
# )

# raw_datasets

print(train)
# for i in train['train']:
#     print(i)
train = DatasetDict({'train': train})
train.map(tokenizer, batched = True, remove_columns = train['train'].column_names)

dict_keys(['Statement', 'tags'])
Dataset({
    features: ['Statement', 'tags'],
    num_rows: 7976
})


KeyError: "Column train not in the dataset. Current columns in the dataset: ['Statement', 'tags']"